# Práctica de Control de sensor Ultrasónico y Cámara.

<br/>
<h5>Se implemento un script de python mantiene el uso de constante sensor ultrsónico y la cámara de la siguiente forma:</h5>
<ol>
<li>En el host maestro hacer siempre</li>
<li>
<ol>
<li> Enviar un pulso por 10 micro segundos</li>
<li> Capturar  el tiempo en segundos que demora en regresar el sonido</li>
<li> Calcular de distancia (tiempo * velocidad del sonido cm/s)/2 </li>
<li> Si la distancia es menor a 200 enviar mensaje al host esclavo </li>


</ol></li>
<li>En el host esclavo hacer siempre</li>

<li>
<ol>
<li> Recibir petición del host maestro</li>
<li> Enviar mensaje de conformación al host maestro</li>
<li> Iniciar la camara y capturar una imagen con el número de la petición enviada</li>
<li> Cerrar la cámara</li>


</ol></li>

</ol>
<br/>
A continuación se muestra el script y cómo se ejecutó en ambos hosts.

In [ ]:
 
import picamera
from mpi4py import MPI
import RPi.GPIO as GPIO
import time as time
import picamera
# Set up GPIO
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
TRIG=23
ECHO=24
PRUEBA=18
inicia_pulso=0
termina_pulso=0

GPIO.setup(PRUEBA, GPIO.OUT) 
GPIO.setup(TRIG, GPIO.OUT) 
GPIO.setup(ECHO, GPIO.IN) 
GPIO.output(TRIG, False) #Seteando transmisor en false


# Inicialización de las  variables MPI : comm, rank, size
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print "Esperando lectura  del sensor" 
time.sleep(1) 


print "rank=",rank
# This is the master node.
if rank == 0:
    request = 0
    process = 1
    while True: 
        #Trigger inicia el pulso
        GPIO.output(TRIG, True)
        #Duracion del pulso 10 micro segundos
        time.sleep(0.00001)
        #Desactiva el pulso
        GPIO.output(TRIG, False)
        #Se guarda el tiempo en segundos
        inicia_pulso = time.time()
    
        #Muentras la recepcion (echo) este en 0,toma el tiempo del inicio del pulso
        while GPIO.input(ECHO)==0:
            inicia_pulso = time.time()
        #Cuando se reciba el rebote del pulso en echo, se toma el tiempo de final del pulso
        while GPIO.input(ECHO)==1:
            termina_pulso = time.time() 
            #Calcular el tiempo que transcurrio 
            duracion  = termina_pulso - inicia_pulso 
        #Calculo de distancia (tiempo * velocidad del sonido cm/s)/2  **porque es ida y vuelta
        distancia = duracion*34300  
        distancia= distancia/2
        #Condicion para envio de mensaje
        if distancia<=200:
            print "Distancia menor es:",distancia,"cm"
            #print "menor de 2 metros"
            GPIO.output(PRUEBA,True)
            time.sleep(.5)
            GPIO.output(PRUEBA,False)
            comm.send(request, dest=1, tag=1)
            print ("se recibe respuesta"+str (comm.recv(source=1,tag=1)))

        time.sleep(0.5) 
        print "Distancia:",distancia,"cm" 
        request+=1
    GPIO.cleanup()


# El nodo esclavo  se mantiene reciviendo mensajes del maestro confirma y captura la foto 
else:
    while True:
        request = comm.recv(source=0, tag=1)
        comm.send(rank, dest=0, tag=1)
        camara = picamera.PiCamera()
        camara.capture("test"+str(request)+".jpg")
        print ("se recibe peticion"+str(request))
        camara.close()
    GPIO.cleanup()

 


 Una vez que se introduce el usuario (pi) y contraseña nos dirigimos a la carpeta donde se encuentra el script.
  <div style="background-color:black; color:green;widht:700px;height:50px">cd comades/python </div>
  
  <br/>
  Copiamos el archivo a <b>rpi-lania4.local</b> con secure copy
  <br/>
  <div style="background-color:black; color:green;widht:700px;height:50px">   scp sensoryCamara.py pi@rpi-lania4.local:~/comades/python </div>

  
  <br/>
  Se modifica el archivo machinefile dejando solo la raspberry esclava (en este caso la 2) y nuestra raspberry.
  Ejecutamos el <b>mpirun</b> para dos procesos, cargando el machine file y el script de python 
  <br/>
  

 <div style="background-color:black; color:green;widht:700px;height:50px">mpirun.openmpi -np 2 -machinefile machinefile python sensorYcamara.py <div/>